In [3]:
import gc
import time
import numpy as np
import pandas as pd
from datetime import datetime

In [4]:
t1 = {'card_id':[1, 2, 1, 3], 
      'A':[1, 2, 1, 2], 
      'B':[2, 1, 2, 2], 
      'C':[4, 5, 1, 5], 
      'D':[7, 5, 4, 8],}

t1 = pd.DataFrame(t1)
t1

,card_id,A,B,C,D
0,1,1,2,4,7
1,2,2,1,5,5
2,1,1,2,1,4
3,3,2,2,5,8


In [5]:
numeric_cols = ['C', 'D']
category_cols = ['A', 'B']

In [6]:
features = {}
card_all = t1['card_id'].values.tolist()
for card in card_all:
    features[card] = {}

In [7]:
features

{1: {}, 2: {}, 3: {}}

In [8]:
columns = t1.columns.tolist()
columns

['card_id', 'A', 'B', 'C', 'D']

In [9]:
idx = columns.index('card_id')
idx

0

In [10]:
category_cols_index = [columns.index(col) for col in category_cols]
category_cols_index

[1, 2]

In [11]:
numeric_cols_index = [columns.index(col) for col in numeric_cols]
numeric_cols_index

[3, 4]

In [12]:
for i in range(t1.shape[0]):
    va = t1.loc[i].values
    card = va[idx]
    for cate_ind in category_cols_index:
        for num_ind in numeric_cols_index:
            col_name = '&'.join([columns[cate_ind], str(va[cate_ind]), columns[num_ind]])
            features[card][col_name] = features[card].get(col_name, 0) + va[num_ind]

In [13]:
features

{1: {'A&1&C': 5, 'A&1&D': 11, 'B&2&C': 5, 'B&2&D': 11},
 2: {'A&2&C': 5, 'A&2&D': 5, 'B&1&C': 5, 'B&1&D': 5},
 3: {'A&2&C': 5, 'A&2&D': 8, 'B&2&C': 5, 'B&2&D': 8}}

In [14]:
df = pd.DataFrame(features).T.reset_index()

cols = df.columns.tolist()

df.columns = ['card_id'] + cols[1:]
df

,card_id,A&1&C,A&1&D,B&2&C,B&2&D,A&2&C,A&2&D,B&1&C,B&1&D
0,1,5.0,11.0,5.0,11.0,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,5.0,5.0,5.0,5.0
2,3,NaN,NaN,5.0,8.0,5.0,8.0,NaN,NaN


# actual data

In [15]:
train = pd.read_csv('preprocess/train_pre.csv')
test =  pd.read_csv('preprocess/test_pre.csv')
transaction = pd.read_csv('preprocess/transaction_d_pre.csv')

In [16]:
numeric_cols = ['purchase_amount', 'installments']

category_cols = ['authorized_flag', 'city_id', 'category_1',
       'category_3', 'merchant_category_id','month_lag','most_recent_sales_range',
                 'most_recent_purchases_range', 'category_4',
                 'purchase_month', 'purchase_hour_section', 'purchase_day']

id_cols = ['card_id', 'merchant_id']

In [38]:
features = {}
card_all = train['card_id'].append(test['card_id']).values.tolist()
for card in card_all:
    features[card] = {}
     
columns = transaction.columns.tolist()
idx = columns.index('card_id')
category_cols_index = [columns.index(col) for col in category_cols]
numeric_cols_index = [columns.index(col) for col in numeric_cols]

s = time.time()
num = 0

for i in range(transaction.shape[0]):
    va = transaction.loc[i].values
    card = va[idx]
    for cate_ind in category_cols_index:
        for num_ind in numeric_cols_index:
            col_name = '&'.join([str(columns[cate_ind]), str(columns[num_ind]), str(va[cate_ind])])
            features[card][col_name] = features[card].get(col_name, 0) + va[num_ind]
    num += 1
    if num%1000000==0:
        print(time.time()-s, "s")
        
del transaction
gc.collect()

C:\Users\Ethan\AppData\Local\Temp\ipykernel_18452\763939360.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  card_all = train['card_id'].append(test['card_id']).values.tolist()


88.29499888420105 s
177.00799894332886 s
265.0792405605316 s
354.3411509990692 s
442.3929316997528 s
528.7509317398071 s
616.4033284187317 s
705.8865299224854 s
794.264880657196 s
882.653906583786 s
970.8641238212585 s
1059.5812394618988 s
1148.7060930728912 s
1237.3886549472809 s
1324.711112499237 s
1413.2174022197723 s
1502.4214012622833 s
1590.2181565761566 s
1678.662453174591 s
1767.8186433315277 s
1856.6575124263763 s
1945.923479795456 s


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(features).T.reset_index()
del features
cols = df.columns.tolist()
df.columns = ['card_id'] + cols[1:]

train = pd.merge(train, df, how='left', on='card_id')
test =  pd.merge(test, df, how='left', on='card_id')
del df
train.to_csv("preprocess/train_dict.csv", index=False)
test.to_csv("preprocess/test_dict.csv", index=False)

gc.collect()

<center><img src="https://i.loli.net/2021/10/23/ZY75eSk3pAayoJn.png" alt="image-20211023161451438" style="zoom:67%;" />

# Group by

In [ ]:
transaction = pd.read_csv('preprocess/transaction_g_pre.csv')

In [ ]:
numeric_cols = ['authorized_flag',  'category_1', 'installments',
       'category_3',  'month_lag','purchase_month','purchase_day','purchase_day_diff', 'purchase_month_diff',
       'purchase_amount', 'category_2', 
       'purchase_month', 'purchase_hour_section', 'purchase_day',
       'most_recent_sales_range', 'most_recent_purchases_range', 'category_4']
categorical_cols = ['city_id', 'merchant_category_id', 'merchant_id', 'state_id', 'subsector_id']

In [ ]:
aggs = {}

for col in numeric_cols:
    aggs[col] = ['nunique', 'mean', 'min', 'max','var','skew', 'sum']
for col in categorical_cols:
    aggs[col] = ['nunique']    
aggs['card_id'] = ['size', 'count']
cols = ['card_id']

for key in aggs.keys():
    cols.extend([key+'_'+stat for stat in aggs[key]])

df = transaction[transaction['month_lag']<0].groupby('card_id').agg(aggs).reset_index()
df.columns = cols[:1] + [co+'_hist' for co in cols[1:]]

df2 = transaction[transaction['month_lag']>=0].groupby('card_id').agg(aggs).reset_index()
df2.columns = cols[:1] + [co+'_new' for co in cols[1:]]
df = pd.merge(df, df2, how='left',on='card_id')

df2 = transaction.groupby('card_id').agg(aggs).reset_index()
df2.columns = cols
df = pd.merge(df, df2, how='left',on='card_id')
del transaction
gc.collect()

train = pd.merge(train, df, how='left', on='card_id')
test =  pd.merge(test, df, how='left', on='card_id')
del df
train.to_csv("preprocess/train_groupby.csv", index=False)
test.to_csv("preprocess/test_groupby.csv", index=False)

gc.collect()

<center><img src="https://i.loli.net/2021/10/23/HpI1QuM6ZvtkS7f.png" alt="image-20211023162707542" style="zoom:67%;" />

# train

In [ ]:
train_dict = pd.read_csv("preprocess/train_dict.csv")
test_dict = pd.read_csv("preprocess/test_dict.csv")
train_groupby = pd.read_csv("preprocess/train_groupby.csv")
test_groupby = pd.read_csv("preprocess/test_groupby.csv")

In [ ]:
for co in train_dict.columns:
    if co in train_groupby.columns and co!='card_id':
        del train_groupby[co]
for co in test_dict.columns:
    if co in test_groupby.columns and co!='card_id':
        del test_groupby[co]

In [ ]:
train = pd.merge(train_dict, train_groupby, how='left', on='card_id').fillna(0)
test = pd.merge(test_dict, test_groupby, how='left', on='card_id').fillna(0)

In [ ]:
train.to_csv("preprocess/train.csv", index=False)
test.to_csv("preprocess/test.csv", index=False)

del train_dict, test_dict, train_groupby, test_groupby
gc.collect()

# Random Forest

In [ ]:
train = pd.read_csv("preprocess/train.csv")
test = pd.read_csv("preprocess/test.csv")

In [ ]:
features = train.columns.tolist()
features.remove("card_id")
features.remove("target")
featureSelect = features[:]

corr = []
for fea in featureSelect:
    corr.append(abs(train[[fea, 'target']].fillna(0).corr().values[0][1]))

se = pd.Series(corr, index=featureSelect).sort_values(ascending=False)
feature_select = ['card_id'] + se[:300].index.tolist()

train = train[feature_select + ['target']]
test = test[feature_select]

### Grid Search

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
features = train.columns.tolist()
features.remove("card_id")
features.remove("target")


parameter_space = {
    "n_estimators": [79, 80, 81], 
    "min_samples_leaf": [29, 30, 31],
    "min_samples_split": [2, 3],
    "max_depth": [9, 10],
    "max_features": ["auto", 80]
}

In [ ]:
clf = RandomForestRegressor(
    criterion="mse",
    n_jobs=15,
    random_state=22)

In [ ]:
grid = GridSearchCV(clf, parameter_space, cv=2, scoring="neg_mean_squared_error")
grid.fit(train[features].values, train['target'].values)

In [ ]:
grid.best_params_


In [ ]:
grid.best_estimator_

In [ ]:
np.sqrt(-grid.best_score_)

In [ ]:
grid.best_estimator_.predict(test[features])